In [77]:
import pandas as pd
import numpy as np
import pickle


    Format de données : 
    CUSTOMER IDENTIFIER | SPECIFIC ITEM IDENTIFIER | GENERAL ITEM IDENTIFIER | hit_date | buy
    
    
    Feats user : is_male , is_female
    Feats article : is_junior, is_male, is_female, eco_design
    
    
    Sports - user :  CUSTOMER IDENTIFIER | sport_id
    
    Sports - item :  SPECIFIC ITEM IDENTIFIER | sport_id
    
    Sports - Sports g : sports_id | sportsgroup_id
    
    Sports feats: osef
    
    Sports one hot : sport_id, 
    

### Préparation des interactions Client - Article

In [79]:
transactions = pd.read_pickle('pickles/transactions')

In [81]:
transactions.head()

,customer_id,article_id,t_dat,price,sales_channel_id,quantity,day_of_week,day_of_year
20523780,b6227fbf6968093cde31b5335a522c36827ed7cc715d01...,685687004,2018-09-20,0.016542,2,1,3,263
1594079,0e129e8643d295d3f00e0d57f55d7d8dff13b82592f590...,700828002,2018-09-20,0.050831,1,1,3,263
17533528,9b871e8d5636a7bad8c826012c7a2b1a04bc93d0075c6e...,685683001,2018-09-20,0.025407,1,1,3,263
4498464,27c273bffa5d3b2558627f2bb5d8de121e2530cd7e2f10...,541519024,2018-09-20,0.016932,1,1,3,263
23254377,ce698d3c10ec286f4aa65cee781ed8a9926eeb3fae330a...,565379023,2018-09-20,0.008458,1,1,3,263


In [82]:
transactions_sampled = transactions.tail(50000)
transactions_sampled.head()

,customer_id,article_id,t_dat,price,sales_channel_id,quantity,day_of_week,day_of_year
19456539,ac933df9b0f55286fc3f547759879edc3f83c2cd24fcd0...,880553002,2020-09-21,0.025407,2,2,0,265
23463746,d04c8253b51d97880d69dca07ab73eecc735f1ddfc811a...,839496002,2020-09-21,0.011847,2,1,0,265
19456541,ac933df9b0f55286fc3f547759879edc3f83c2cd24fcd0...,894757001,2020-09-21,0.030492,2,1,0,265
10184553,5a769171a2a9b5a13228ae5edb308c0ad81921cbdc233a...,677930037,2020-09-21,0.015237,2,1,0,265
19456544,ac933df9b0f55286fc3f547759879edc3f83c2cd24fcd0...,903910001,2020-09-21,0.084729,2,1,0,265


In [83]:
transactions_sampled = transactions_sampled[['customer_id', 'article_id', 't_dat']]

In [84]:
transactions_sampled.rename({
    'customer_id': 'CUSTOMER IDENTIFIER',
    'article_id': 'SPECIFIC ITEM IDENTIFIER',
    't_dat': 'hit_date'
}, axis=1, inplace = True)

In [85]:
transactions_sampled['buy'] = 1

In [86]:
transactions_sampled.head()

,CUSTOMER IDENTIFIER,SPECIFIC ITEM IDENTIFIER,hit_date,buy
19456539,ac933df9b0f55286fc3f547759879edc3f83c2cd24fcd0...,880553002,2020-09-21,1
23463746,d04c8253b51d97880d69dca07ab73eecc735f1ddfc811a...,839496002,2020-09-21,1
19456541,ac933df9b0f55286fc3f547759879edc3f83c2cd24fcd0...,894757001,2020-09-21,1
10184553,5a769171a2a9b5a13228ae5edb308c0ad81921cbdc233a...,677930037,2020-09-21,1
19456544,ac933df9b0f55286fc3f547759879edc3f83c2cd24fcd0...,903910001,2020-09-21,1


In [87]:
transactions_sampled.to_pickle("pickles/gnn_user_item.pkl")

### Préparation des features client

In [88]:
customers = pd.read_pickle('pickles/customers.pkl')

In [90]:
# Rename columns.
customers.rename({
    "customer_id": 'CUSTOMER IDENTIFIER',
    "FN": 'is_male',
    "fashion_news_frequency": "is_female"
}, axis = 1, inplace = True)

In [91]:
# Selecting only customers that are mentionned in user_item DataFrame.
user_items = pd.read_pickle("pickles/gnn_user_item.pkl")
user_ids = pd.Series(user_items['CUSTOMER IDENTIFIER'].unique())
customers = customers.merge(user_ids.rename('CUSTOMER IDENTIFIER'), how = "inner", on = "CUSTOMER IDENTIFIER")

In [92]:
customers = customers[['CUSTOMER IDENTIFIER', 'is_male', 'is_female']]

In [93]:
customers['is_female'] = np.where(customers['is_female'] == 'Regularly', 1, 0) 

In [94]:
customers['is_male'] = customers['is_male'].astype(int)

In [96]:
customers.to_pickle("pickles/gnn_user_features.pkl")
customers.head()

,CUSTOMER IDENTIFIER,is_male,is_female
0,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,1,1
1,000525e3fe01600d717da8423643a8303390a055c578ed...,1,1
2,0010e8eb18f131e724d6997909af0808adbba057529edb...,1,1
3,001436e2c83cda28548dd668cfc7d621d70d2baf6f6cf0...,0,0
4,0026ebdd70715d8fa2befa14dfed317a1ffe5451aba839...,1,1


In [100]:
customer_sports = pd.DataFrame(columns = ['CUSTOMER IDENTIFIER', 'sport_id'])
customer_sports.to_pickle("pickles/gnn_user_sport.pkl")
customer_sports.head()

,CUSTOMER IDENTIFIER,sport_id


### Préparation des features Article et des Catégories

In [120]:
articles = pd.read_pickle('pickles/articles')

In [121]:
articles.rename({
    "article_id": 'SPECIFIC ITEM IDENTIFIER',
}, axis = 1, inplace = True)

In [123]:
user_items = pd.read_pickle("pickles/gnn_user_item.pkl")
item_ids = pd.Series(user_items['SPECIFIC ITEM IDENTIFIER'].unique()).rename('SPECIFIC ITEM IDENTIFIER')
articles = articles.merge(item_ids, how = "inner", on = "SPECIFIC ITEM IDENTIFIER")

In [124]:
articles['GENERAL ITEM IDENTIFIER'] = articles['product_code']

In [125]:
articles[['index_group_no', 'index_group_name']].value_counts()
articles['is_junior'] = np.where(articles['index_group_no'] == 4, 1, 0)
articles['is_male'] = np.where(articles['index_group_no'] == 3, 1, 0)
articles['is_female'] = np.where(articles['index_group_no'] == 1, 1, 0)
articles['eco_design'] = np.where(articles['index_group_no'] == 2, 1, 0)

In [126]:
item_feats = articles[['SPECIFIC ITEM IDENTIFIER', 'GENERAL ITEM IDENTIFIER', 'is_junior', 'is_male', 'is_female', 'eco_design']]
item_feats.to_pickle("pickles/gnn_item_features.pkl")
item_feats.head()

,SPECIFIC ITEM IDENTIFIER,GENERAL ITEM IDENTIFIER,is_junior,is_male,is_female,eco_design
0,111565001,111565,0,0,1,0
1,111586001,111586,0,0,1,0
2,111593001,111593,0,0,1,0
3,111609001,111609,0,0,1,0
4,123173001,123173,0,0,1,0


In [127]:
# Extraction des catégories
categories_group = articles[['product_type_no', 'product_group_name']].groupby("product_type_no",  as_index=False).agg({'product_group_name': 'first'})

categories_group.rename({
    'product_type_no': 'sports_id',
    'product_group_name': 'sportsgroup_id'
}, axis =1, inplace = True)
categories_group.head()

from sklearn.preprocessing import LabelEncoder

# Encodage du groupe
categories_group['sportsgroup_id'] = LabelEncoder().fit_transform(categories_group['sportsgroup_id'])

categories_group.to_pickle("pickles/gnn_sport_groups.pkl")
categories_group.head()

,sports_id,sportsgroup_id
0,-1,14
1,49,13
2,57,11
3,59,11
4,66,0


In [130]:
categories_group = pd.DataFrame(columns = ['sports_id', 'sportsgroup_id'])
categories_group.to_pickle("pickles/gnn_sport_groups.pkl")
categories_group.head()

,sports_id,sportsgroup_id


In [108]:
item_categories = articles[['product_type_no', 'SPECIFIC ITEM IDENTIFIER']]
item_categories.rename({
    'product_type_no': 'sport_id',
}, axis =1, inplace = True)

item_categories.to_pickle("pickles/gnn_item_sport.pkl")
item_categories.head()

/home/treb/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,sport_id,SPECIFIC ITEM IDENTIFIER
0,304,111565001
1,273,111586001
2,304,111593001
3,304,111609001
4,273,123173001


In [113]:
sports_ids = pd.Series(item_categories['sport_id'].unique())
sports = pd.get_dummies(sports_ids)
sports['sport_id'] = sports_ids

sports.to_pickle("pickles/gnn_sports.pkl")

In [115]:
sports.shape

(103, 104)

In [117]:
item_categories['sport_id'].value_counts().shape

(103,)

(103,)